### Training the model to select put strategy

### Train Stock Price

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Load the historical options data with simulated trade outcomes.
# Each row represents one trade opportunity for a specific out-of-the-money (OTM) put.
df = pd.read_csv(r"C:\Users\Sezy\OneDrive\Personal_Projects\Trading_Algorithm\CSVFILES\historical_stock_data_with_premiums.csv", parse_dates=["Date"])

# Simulate the trade: look 3 weeks ahead (approx. 15 trading days).
df["Future_Price"] = df.groupby("Ticker")["Stock Price"].shift(-15)
df = df.dropna(subset=["Future_Price"])

# Expand the dataset: one row per trade opportunity per available OTM strike (OTM1-OTM4)
expanded_rows = []
contract_size = 100  # Standard contract size

for otm in range(1, 5):
    for idx, row in df.iterrows():
        new_row = row.copy()
        new_row["StrikePrice"] = row[f"OTM{otm}"]
        new_row["Premium"] = row[f"Pred_Premium_OTM{otm}"]
        # Calculate profit:
        # If the option expires worthless (Future_Price > StrikePrice), profit = premium * contract size.
        # If the option is exercised (Future_Price <= StrikePrice), profit = premium minus the loss from assignment.
        if row["Future_Price"] > row[f"OTM{otm}"]:
            profit = row[f"Pred_Premium_OTM{otm}"] * contract_size
        else:
            profit = row[f"Pred_Premium_OTM{otm}"] * contract_size - ((row[f"OTM{otm}"] - row["Future_Price"]) * contract_size)
        new_row["Profit"] = profit
        new_row["OTM_Level"] = f"OTM{otm}"
        expanded_rows.append(new_row)

df_expanded = pd.DataFrame(expanded_rows)

# Features we use:
# - Current Stock Price
# - 50-day and 200-day moving averages (sma_50, sma_200)
# - Annualized volatility
# - The chosen Strike Price
# - The option's Premium
feature_cols = ["Stock Price", "sma_50", "sma_200", "annualized_volatility", "StrikePrice", "Premium"]
df_expanded = df_expanded.dropna(subset=feature_cols + ["Profit"])
X = df_expanded[feature_cols]
y = df_expanded["Profit"]

# Standardize the features.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a RandomForestRegressor to predict the profit from selling a put.
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the trained model and scaler.
joblib.dump(model, "trained_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Model training complete. Files saved as trained_model.pkl and scaler.pkl")

Model training complete. Files saved as trained_model.pkl and scaler.pkl
